In [2]:

import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
csv_file_path = 'data_2.csv'  # Replace with the path to your CSV file
df = pd.read_csv(csv_file_path)
df = df.sort_values(by='TimeUS')

In [4]:
df.head()

,Seq,Stamp,TimeUS,Roll,Pitch,Yaw,Alt,Lat,Lng,C1_Roll,C2_Pitch,C3_Throttle
0,198.0,17:34.2,179654476.0,45.08,-5.59,61.29,-12.06,22.555598,88.308273,1500.0,1500.0,987.0
1,423.0,17:34.2,179697878.0,42.66,-4.62,60.23,-12.07,22.555598,88.308274,1500.0,1500.0,987.0
2,649.0,17:34.3,179733964.0,40.29,-3.70,59.38,-12.07,22.555598,88.308274,1500.0,1500.0,987.0
3,879.0,17:34.3,179774486.0,37.83,-2.72,58.68,-12.07,22.555598,88.308274,1500.0,1500.0,987.0
4,1109.0,17:34.4,179841066.0,35.23,-1.15,58.16,-12.07,22.555598,88.308274,1500.0,1500.0,987.0


In [5]:
df['Roll_diff'] = df['Roll'].shift(-1) - df['Roll']
df['Pitch_diff'] = df['Pitch'].shift(-1) - df['Pitch']
df['Yaw_diff'] = df['Yaw'].shift(-1) - df['Yaw']
df['Alt_diff'] = df['Alt'].shift(-1) - df['Alt']
df['Lat_diff'] = df['Lat'].shift(-1) - df['Lat']
df['Lng_diff'] = df['Lng'].shift(-1) - df['Lng']
df.head()

,Seq,Stamp,TimeUS,Roll,Pitch,Yaw,Alt,Lat,Lng,C1_Roll,C2_Pitch,C3_Throttle,Roll_diff,Pitch_diff,Yaw_diff,Alt_diff,Lat_diff,Lng_diff
0,198.0,17:34.2,179654476.0,45.08,-5.59,61.29,-12.06,22.555598,88.308273,1500.0,1500.0,987.0,-2.42,0.97,-1.06,-0.01,-2.000000e-07,1.900000e-06
1,423.0,17:34.2,179697878.0,42.66,-4.62,60.23,-12.07,22.555598,88.308274,1500.0,1500.0,987.0,-2.37,0.92,-0.85,0.00,0.000000e+00,0.000000e+00
2,649.0,17:34.3,179733964.0,40.29,-3.70,59.38,-12.07,22.555598,88.308274,1500.0,1500.0,987.0,-2.46,0.98,-0.70,0.00,0.000000e+00,0.000000e+00
3,879.0,17:34.3,179774486.0,37.83,-2.72,58.68,-12.07,22.555598,88.308274,1500.0,1500.0,987.0,-2.60,1.57,-0.52,0.00,0.000000e+00,0.000000e+00
4,1109.0,17:34.4,179841066.0,35.23,-1.15,58.16,-12.07,22.555598,88.308274,1500.0,1500.0,987.0,-1.35,-0.10,-0.56,0.45,5.000000e-07,8.000000e-07


In [6]:
train_input_columns = ['Roll', 'Pitch', 'Yaw', 'Alt', 'Lat', 'Lng', 'Roll_diff', 'Pitch_diff', 'Yaw_diff', 'Alt_diff', 'Lat_diff', 'Lng_diff']
X = df[train_input_columns]
train_output_columns = ['C1_Roll', 'C2_Pitch', 'C3_Throttle']
Y = df[train_output_columns]
print(Y['C3_Throttle'].max())
print(Y['C3_Throttle'].min())

1848.0
987.0


In [7]:
#Normalize 
#Assumptions are as follows
# roll, pitch, yaw -> 0-90
#lat -> -90 to 90
#long -> -180 to 180
#c1, c2, c3 -> 500-2500
#Alt -50 to +50 #TODO:Unsure 
Y_norm = Y.copy()
Y_norm[train_output_columns] = (Y[train_output_columns] - 500)/(2500-500)
X_norm = X.copy()
columns = ['Roll', 'Pitch', 'Yaw', 'Roll_diff', 'Pitch_diff', 'Yaw_diff']
X_norm[columns] = (X[columns] - 0)/(90-0)
X_norm[['Alt','Alt_diff']] = (X[['Alt','Alt_diff']] - (-50))/(50-(-50))
X_norm[['Lat', 'Lat_diff']] = (X[['Lat', 'Lat_diff']] - (-90))/(90-(-90))
X_norm[['Lng', 'Lng_diff']] = (X[['Lng', 'Lng_diff']] - (-180))/(180-(-180))


In [8]:
X_norm = X_norm[:1]
Y_norm = Y_norm[:1]
X_norm.shape

(1, 12)

In [9]:
print(X_norm.isna())
print(Y_norm.isna())

    Roll  Pitch    Yaw    Alt    Lat    Lng  Roll_diff  Pitch_diff  Yaw_diff  \
0  False  False  False  False  False  False      False       False     False   

   Alt_diff  Lat_diff  Lng_diff  
0     False     False     False  
   C1_Roll  C2_Pitch  C3_Throttle
0    False     False        False


In [11]:
class Neural_net():
    def __init__(self, input_shape, output_shape):
        self.create_model(input_shape, output_shape)

    def create_model(self, input_shape, output_shape):
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(output_shape, input_dim = 12,activation='sigmoid'),
        ])
        self.model.compile(loss = 'MSE',
               optimizer = 'adam')
        self.model.summary()

    def train(self, x, y, num_epochs):
        #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
        #print(X_train.head())
        #print(y_train.head())
        print(self.model.predict(x))
        self.model.fit(x,y, 10)
        print(self.model.evaluate(x, y))
        #print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')
print(X.shape[-1])
print(Y.shape[-1])
nn1 = Neural_net(X_norm.shape[-1], Y_norm.shape[-1])
nn1.train(X_norm, Y_norm, 10)

12
3
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 3)                 39        
                                                                 
Total params: 39 (156.00 Byte)
Trainable params: 39 (156.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
1/1 [==============================] - 0s 47ms/step
[[0.5333843  0.6147693  0.44344774]]
1/1 [==============================] - 0s 65ms/step - loss: 0.0178
0.017776956781744957
